In [1]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, LearningRateScheduler, TensorBoard
from keras import backend as K
from keras.callbacks import History, ModelCheckpoint
from keras.models import load_model
from ssd300 import build_SSD300
# from parser import Parser
import matplotlib.pyplot as plt
import numpy as np
import os
from load_pred import load_img, resize_im
from metrics import iou_metrics
from Decoder import Decoder, batch_decoder, decode_output
# from Encoder import Encoder, encode_batch
from box_utils import IoU, generate_default_boxes
from loss_function import loss_function
from keras import metrics, losses
import h5py
from pycocotools.coco import COCO

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
# Configuration of the model: 
numClasses = 70
input_shape = (300, 300, 3)
nms_thres=0.0 # IoU threshold for non-maximal suppression
score_thres=0.01 # threshold for classification scores
top_k=200 # the maximum number of predictions kept per image
map_size=[38, 19, 10, 5, 3, 1]
min_scale=0.2 # the smallest scale of the feature map
max_scale=0.9 # the largest scale of the feature map
aspect_ratios=[0.5, 1, 2] # aspect ratios of the default boxes to be generated
n_layers=6 # the number of prediction blocks
prediction_size=[38, 19, 10, 5, 3, 1] # sizes of feature maps at each level
fileName = "pred_img.txt"
model_path = os.getcwd()
model_path = os.path.join(model_path, 'weights.04-0.38.hdf5')

im_collection = load_img(fileName)

In [3]:
K.clear_session() # Clear previous session

defaults = generate_default_boxes(n_layers=n_layers, 
                                min_scale=min_scale, 
                                max_scale=max_scale, 
                                map_size=prediction_size, 
                                aspect_ratios=aspect_ratios)

In [4]:
# Create a metrics function
def iou(Y_true, Y_pred): 
    return iou_metrics(Y_true, Y_pred, defaults)

model = load_model(model_path, custom_objects={'loss_function': loss_function, 'iou': iou})
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss=loss_function, metrics=['accuracy', iou])


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
n_pred = im_collection.shape[0]
pred_data = resize_im(im_collection, input_shape=input_shape)
pred = model.predict(pred_data)

In [6]:
decoded_pred = batch_decoder(pred, 
                            numClasses=numClasses,
                            n_layers=n_layers, 
                            min_scale=min_scale, 
                            max_scale=max_scale, 
                            nms_thres=0, 
                            score_thres=score_thres, 
                            top_k=top_k, 
                            aspect_ratios=aspect_ratios)

In [8]:
colors = plt.cm.hsv(np.linspace(0, 1, numClasses+1)).tolist()
data_dir = "/Users/tranle/mscoco"
data_type = "val2017"
annFile = '{}/annotations/instances_{}.json'.format(data_dir,data_type)
coco = COCO(annFile)
cats = coco.loadCats(coco.getCatIds())
classes = [cat['name'] for cat in cats][:numClasses]
print(classes)


# for i in range(n_pred): 
#     curr = im_collection[i]
#     im_shape = curr.shape
#     curr_pred = decoded_pred[i]
#     plt.figure(figsize=(20,12))
#     plt.imshow(curr)
#     current_axis = plt.gca()
#     n_boxes = curr_pred.shape[0]
#     print(n_boxes)
#     for j in range(n_boxes): 
#         curr_box = curr_pred[j]
#         xmin = curr_box[-4]*im_shape[0]
#         ymin = curr_box[-3]*im_shape[1]
#         xmax = curr_box[-2]*im_shape[0]
#         ymax = curr_box[-1]*im_shape[1]
#         label_id = curr_box[0]# Find the correct class where it is not zero
#         score = curr_box[1]
#         color = colors[int(label_id)]
#         label = '{}: {:.2f}'.format(classes[int(label_id)], score)
#         current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
#         current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})

loading annotations into memory...
Done (t=0.66s)
creating index...
index created!
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven']
